### Data cleanning on anime & anime_with_synopsis

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from surprise import dump
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from IPython.display import display
import ipywidgets as widgets
from surprise.model_selection import GridSearchCV

from proj_util import *

In [25]:

PATH = './Data'

anime = pd.read_csv(f'{PATH}/anime.csv')
anime_with_synopsis = pd.read_csv(f'{PATH}/anime_with_synopsis.csv')

In [26]:
replace_dict = {'Unknown': np.nan}
anime_with_synopsis = anime_with_synopsis.replace(replace_dict)
anime = anime.replace(replace_dict)

change genre into itemset

In [27]:
anime_with_synopsis['Genres'] = anime_with_synopsis['Genres'].apply(lambda x: set((str(x)).strip().split(', ')))
anime['Genres'] = anime['Genres'].apply(lambda x: set((str(x)).strip().split(', ')))

In [28]:
genres = set()
for i in anime_with_synopsis['Genres'].to_numpy():
    genres = genres.union(i)

In [29]:
genres

{'Action',
 'Adventure',
 'Cars',
 'Comedy',
 'Dementia',
 'Demons',
 'Drama',
 'Ecchi',
 'Fantasy',
 'Game',
 'Harem',
 'Historical',
 'Horror',
 'Josei',
 'Kids',
 'Magic',
 'Martial Arts',
 'Mecha',
 'Military',
 'Music',
 'Mystery',
 'Parody',
 'Police',
 'Psychological',
 'Romance',
 'Samurai',
 'School',
 'Sci-Fi',
 'Seinen',
 'Shoujo',
 'Shoujo Ai',
 'Shounen',
 'Shounen Ai',
 'Slice of Life',
 'Space',
 'Sports',
 'Super Power',
 'Supernatural',
 'Thriller',
 'Vampire',
 'Yaoi',
 'nan'}

In [ ]:
anime_with_synopsis = anime_with_synopsis.replace({'sypnopsis': '^No synopsis information.*$'}, {'sypnopsis': ''}, regex=True)

In [ ]:
display(anime_with_synopsis[anime_with_synopsis['Genres'] == {'nan'}])
display(anime[anime['Genres'] == {'nan'}])

In [ ]:
anime_with_synopsis['Genres'] = anime_with_synopsis['Genres'].apply(lambda x: x - {'nan'})
anime['Genres'] = anime['Genres'].apply(lambda x: x - {'nan'})

In [ ]:
anime = anime.astype({k: 'float' for k in ['Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1']})

In [ ]:
anime['English name'] = anime['English name'].apply(clean_string)
anime_with_synopsis['Name'] = anime_with_synopsis['Name'].apply(clean_string)

factor premired date into the year

In [ ]:
def get_year_from_string(season_str: str):
    '''Convert string of premired year into year'''
    r = re.compile('(\w+)\W*(\d+)')
    if type(season_str) != str:
        return np.nan 
    splitted = r.findall(season_str)
    if len(splitted) != 1:
        return np.nan

    return int(splitted[0][1])

In [ ]:
anime['Premiered Year'] = anime['Premiered'].apply(get_year_from_string)

In [ ]:
anime['Premiered Year']

#### for seachable title

In [ ]:
anime_for_search = pd.DataFrame({
    'MAL_ID': anime['MAL_ID'],
    'Name': anime['Name'], 
    'English name': anime['English name'],
    'Japanese name': anime['Japanese name'],
    'Genres': anime['Genres'],
    'Avg. Score': anime['Score']
    })

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorizer_en = TfidfVectorizer(ngram_range=(1, 2))
tfidf = vectorizer.fit_transform(anime_for_search['Name'])
tfidf_en = vectorizer_en.fit_transform(anime_for_search['English name'])

In [ ]:

with open('./Model/search_TfidfVectorizer.pkl', '+wb') as f:
    pickle.dump(vectorizer, f)
with open('./Model/search_TfidfVectorizer_en.pkl', '+wb') as f:
    pickle.dump(vectorizer_en, f)

with open('./Data/search_tfidf.pkl', '+wb') as f:
    pickle.dump(tfidf, f)
with open('./Data/search_tfidf_en.pkl', '+wb') as f:
    pickle.dump(tfidf_en, f)

In [ ]:
# save file
anime_with_synopsis.to_feather('./Data/anime_with_synopsis.feather')
anime.to_feather('./Data/anime.feather')
anime_for_search.to_feather('./Data/anime_for_search.feather')

In [3]:
with open('./Model/search_TfidfVectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
    assert(isinstance(vectorizer, TfidfVectorizer))
with open('./Model/search_TfidfVectorizer_en.pkl', 'rb') as f:
    vectorizer_en = pickle.load(f)
    assert(isinstance(vectorizer_en, TfidfVectorizer))

with open('./Data/search_tfidf.pkl', 'rb') as f:
    tfidf = pickle.load(f)
with open('./Data/search_tfidf_en.pkl', 'rb') as f:
    tfidf_en = pickle.load(f)

anime_with_synopsis = pd.read_feather('./Data/anime_with_synopsis.feather')
anime = pd.read_feather('./Data/anime.feather')
anime_for_search = pd.read_feather('./Data/anime_for_search.feather')

def search(keyword: str):
    return search_util(keyword, vectorizer, vectorizer_en, tfidf, tfidf_en, anime_for_search)

In [4]:
search('One piece')

,MAL_ID,Name,English name,Japanese name,Genres,Avg. Score
11,21,One Piece,One Piece,ONE PIECE,"[Adventure, Action, Shounen, Comedy, Fantasy, ...",8.52
6842,12859,One Piece Film: Z,One Piece Film Z,ワンピース フィルム ﻿Ｚ,"[Adventure, Action, Shounen, Comedy, Fantasy, ...",8.18
5260,8171,One Piece Recap,One Piece Recap,ワンピース,"[Adventure, Action, Shounen, Comedy, Fantasy, ...",7.16
7385,16143,One Piece: Kinkyuu Kikaku One Piece Kanzen Kou...,"One Piece:Emergency Planning, A Perfect Strate...",緊急企画ワンピース完全攻略法,"[Comedy, Adventure, Shounen, Fantasy]",7.15
14553,37902,One Piece: Episode of Sorajima,One Piece:Episode of Skypiea,ONE PIECE エピソードオブ空島,"[Adventure, Action, Shounen, Comedy, Fantasy, ...",7.11
433,462,One Piece Movie 4: Dead End no Bouken,One Piece:Dead End,ワンピース デッドエンドの冒険,"[Adventure, Action, Shounen, Comedy, Fantasy, ...",7.59
3524,4155,One Piece Film: Strong World,One Piece Film Strong World,ワンピース フィルム ストロングワールド,"[Adventure, Action, Shounen, Comedy, Fantasy, ...",8.17
13502,36240,Scratch x One Piece Film: Gold,Scratch x One Piece Film:Gold,SCRATCH × ONE PIECE FILM GOLD,"[Fantasy, Shounen]",6.14
4048,5252,One Piece: Romance Dawn Story,One Piece:Romance Dawn Story,ワンピース ロマンスドーンストーリー,"[Action, Shounen, Comedy, Fantasy, Super Power]",7.39
431,460,One Piece Movie 2: Nejimaki-jima no Daibouken,One Piece:Clockwork Island Adventure,ワンピース ねじまき島の冒険,"[Adventure, Action, Shounen, Comedy, Fantasy, ...",7.17


In [ ]:
anime[anime['Genres'].apply(lambda x: 'Hentai' in x) | anime['Score'].isna()].shape

### Data cleanning on Ratings

In [ ]:
# rating = pd.read_csv('./Data/animelist.csv')
# rating_complete = pd.read_csv('./Data/rating_complete.csv')
# rating.to_feather('./Data/animelist.feather')
# rating_complete.to_feather('./Data/rating_complete.feather')

After save as feather binary file, load this instead

In [7]:
rating = pd.read_feather('./Data/animelist.feather')
rating_complete = pd.read_feather('./Data/rating_complete.feather')

In [ ]:
rating['rating'].unique()

I think the rating on `anime_id` is refer to `MAL_ID`

In [ ]:
anime_ids = rating_complete['anime_id'].unique()
np.setdiff1d(anime_ids, anime['MAL_ID'].to_numpy())

#### Just try to add count into the anime file

#### EDA (Ratings)
Checking how many reviews for each anime, it appears that there are some anime which is so popular that there are more than 100,000 reviews on them.

In [ ]:
rating_hist = rating_complete.loc[:, ['anime_id', 'rating']].groupby(by='anime_id').count()

In [ ]:
rating_hist[rating_hist['rating'] < 2e4].hist()

In [ ]:
rating_hist[rating_hist['rating'] > 1e5].hist()

Join the with rating with synopsis

In [ ]:
anime_with_synopsis_with_count = rating_hist.join(anime_with_synopsis.set_index('MAL_ID'))

In [ ]:
anime_with_synopsis_with_count.columns = ['rating_completed_count', 'Name', 'Score', 'Genres', 'sypnopsis']

In [ ]:
anime_with_synopsis_with_count.sort_values(by='rating_completed_count', ascending=False).head(10)

### Recommendation System (SVD)

Partition the dataset for performance reason

In [8]:
user_ids = rating_complete['user_id'].unique()

In [16]:
user_ids.shape

(310059,)

In [17]:
size = 0.05
focus_user_id, _ = train_test_split(user_ids, train_size=0.05, random_state=42)
focus_rating = rating_complete[rating_complete['user_id'].isin(focus_user_id)]

In [ ]:
print(focus_user_id.shape)
print(focus_rating.shape)

In [ ]:
focus_rating.shape[0] / focus_user_id.shape[0]

In [18]:
reader = Reader(rating_scale=(0, 10))
dataset = Dataset.load_from_df(focus_rating[['user_id', 'anime_id', 'rating']], reader)

In [ ]:
# svd = SVD()
# cross_validate(svd, dataset, measures=['RMSE', 'MAE'], cv=3, verbose=True)

```
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.1765  1.1749  1.1763  1.1759  0.0007  
MAE (testset)     0.8790  0.8777  0.8789  0.8785  0.0006  
Fit time          12.93   12.49   12.95   12.79   0.21    
Test time         6.14    6.15    6.26    6.18    0.05    
{'test_rmse': array([1.1765492 , 1.17493892, 1.17632392]),
 'test_mae': array([0.87901695, 0.87771758, 0.87887208]),
 'fit_time': (12.930427074432373, 12.493419647216797, 12.945121049880981),
 'test_time': (6.144198894500732, 6.145112037658691, 6.2577879428863525)}
```

In [ ]:
# knn = KNNBasic()
# cross_validate(knn, dataset, cv=3, verbose=True)

```
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.2941  1.2918  1.2924  1.2928  0.0010  
MAE (testset)     0.9587  0.9581  0.9584  0.9584  0.0002  
Fit time          190.73  171.00  173.67  178.47  8.74    
Test time         598.17  549.93  575.58  574.56  19.71   
{'test_rmse': array([1.29408973, 1.29182792, 1.29243483]),
 'test_mae': array([0.95870405, 0.95810879, 0.9584189 ]),
 'fit_time': (190.73062705993652, 170.99515986442566, 173.67348313331604),
 'test_time': (598.1731100082397, 549.9259850978851, 575.5787467956543)}
```

In [ ]:
# sim_options = sim_options = {
#     "name": "cosine",
#     "user_based": False,  # compute  similarities between items
# }
# knn = KNNBasic(sim_options=sim_options)
# cross_validate(knn, dataset, cv=3, verbose=True)

```
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.3754  1.3752  1.3734  1.3747  0.0009  
MAE (testset)     1.0304  1.0302  1.0292  1.0299  0.0005  
Fit time          88.27   103.83  104.06  98.72   7.39    
Test time         159.00  161.31  167.64  162.65  3.65    
{'test_rmse': array([1.37540845, 1.37515701, 1.37341346]),
 'test_mae': array([1.03036109, 1.0302266 , 1.02920084]),
 'fit_time': (88.27486371994019, 103.82556104660034, 104.05858612060547),
 'test_time': (159.00390911102295, 161.3134388923645, 167.64093279838562)}
```

In [ ]:
# trainset = dataset.build_full_trainset()
# svd.fit(trainset)

In [ ]:
# dump.dump('./Model/svd.pkl', svd)

Model selection

In [ ]:
# param_grid = {
#   'n_factors': [20, 50, 100],
#   'n_epochs': [5, 10, 20]
# }
 
# gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
# gs.fit(dataset)
 
# print(gs.best_score['rmse'])
# print(gs.best_params['rmse'])

```
1.1458925132330084
{'n_factors': 20, 'n_epochs': 20}
```

In [19]:
svd = SVD(n_factors=20, n_epochs=20)
svd.fit(dataset.build_full_trainset())
dump.dump('./Model/svd.pkl', svd)

In [ ]:
focus_rating[focus_rating['user_id'] == focus_user_id[100]].set_index('anime_id').join(anime_for_search.set_index('MAL_ID'))
x = search('Kara no Kyoukai 4')
x['Predict Score'] = x['MAL_ID'].apply(lambda id: svd.predict(uid=focus_user_id[0], iid=id).est)
x

In [ ]:
focus_rating.reset_index().to_feather('./Data/For_SVD/focus_rating.feather')

### Actual Model for recommender system

In [10]:
size = 0.2
focus_user_id_2, _ = train_test_split(user_ids, train_size=size, random_state=42)
focus_rating_2 = rating_complete[rating_complete['user_id'].isin(focus_user_id_2)]
focus_rating_2 = focus_rating_2.reset_index()
focus_rating_2.reset_index().to_feather('./Data/For_SVD/focus_rating_2.feather')

In [11]:
print(f'# of users: {len(focus_user_id_2)}')
print(f'# of ratings: {len(focus_rating_2)}')
print(f'# anime titles: {len(anime_for_search)}')
print(f'sparce ratio: {len(focus_rating_2) / (len(focus_user_id_2) * len(focus_rating_2))}')

# of users: 62011
# of ratings: 11474169
# anime titles: 17562
sparce ratio: 1.6126171163180727e-05


In [13]:
svd_big = SVD(n_factors=20, n_epochs=20)
reader_big = Reader(rating_scale=(0, 10))
dataset_big = Dataset.load_from_df(focus_rating_2[['user_id', 'anime_id', 'rating']], reader_big)
svd_big.fit(dataset_big.build_full_trainset())

In [15]:
dump.dump('./Model/svd_big.pkl', svd_big)

### Interactive session

In [ ]:
search_widget = widgets.Text(
    value = '', description = 'Title'
)

search_btn = widgets.Button(
    description='Search',
    disabled=False,
    button_style='info',
    tooltip='Search',
    icon='search' 
)

search_output = widgets.Output()

def search_event(sender):
    search_output.clear_output()
    x = search_widget.value
    if len(x) < 3:
            return
    with search_output:
        display(search(x))

search_btn.on_click(search_event)

display(widgets.VBox([widgets.HBox([search_widget, search_btn]), search_output]))

In [23]:
from sklearn.metrics import jaccard_score
def search_similar_users(given_review: dict, top_k = 1):
    rank = pd.Series()
    v = pd.DataFrame({'anime_id': given_review.keys(), 'rating': given_review.values()})\
        .pivot(columns='anime_id', values='rating')
    for i in user_ids[:2000]:
        u = focus_rating_2[focus_rating_2['user_id'] == i].pivot(index='user_id', columns='anime_id', values='rating')
        cos_sim = cosine_similarity(pd.concat([u, v]).fillna(0))[0][1]

        rank[i] = cos_sim
        if len(rank) > top_k:
            rank.sort_values(inplace=True, ascending=False)
            rank = rank[:top_k]

    return rank

a = {235: 8, 21:10}
x = pd.DataFrame({'anime_id': a.keys(), 'rating':a.values()})
search_similar_users(a, 3)

0    0.0
1    0.0
2    0.0
dtype: float64